In [1]:
import math
import datetime, time
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import cumsum, log, polyfit, sqrt, std, subtract
from numpy.random import randn

In [2]:
def hurst(ts):
    npts = np.array(ts)
    logs = log(npts[:-1]/npts[1:])
    groups = np.arange(int(len(logs) / 10), 1, -1)
    RS = np.zeros(len(groups)+1)
    line = np.zeros(len(groups)+1)
    for group in groups:
        sub_RS = np.zeros(group)
        stdev = np.zeros(group)
        line[group-1] = len(logs)/group
        for sub in range(group):
            RSmax, RSmin = 0, 0
            sub_group = np.array(logs[int(len(logs)/group)*sub : int(len(logs)/group*(sub+1))])
            for in_sub in range(1, int(len(logs)/group)):
                sub_RS[sub] += sub_group[in_sub]
                RSmax = max(sub_RS[sub], RSmax)
                RSmin = min(sub_RS[sub], RSmin)
            sub_RS[sub] = RSmax - RSmin
            stdev[sub] = std(sub_group)
        a = sub_RS / stdev
        RS[group-1] = a.mean()
    poly = polyfit(log(line[1:]), log(RS[1:]), 1)
    H = poly[0] 
    return H 

In [3]:
def getparams(ticker, strt, endt, intrvl):
    format = '%Y-%m-%d'
    dtstrt = datetime.datetime.strptime(strt, format)
    dtendt = datetime.datetime.strptime(endt, format)
    time_delta = dtendt-dtstrt
    time_marshall_as_year = time_delta.total_seconds() / 365.25 / 86400
    print("year frac = ", time_marshall_as_year)
    data = yf.download(ticker, start=strt, end=endt, interval=intrvl)
    tau = time_marshall_as_year / len(data['Adj Close'])
    print("tau = ", tau)
    ticks = np.arange(1,len(data['Adj Close'])+1)
    print(len(ticks))
    #plt.plot(ticks, data['Adj Close'])
    #plt.show()
    np_data = data['Adj Close']
    log_data = np.zeros(len(np_data))
    div = np_data[0]
    log_data = np.log(np_data / div)
    XT = log_data[len(log_data)-1]
    mean = XT / len(log_data)
    var = 0
    for i in range(1, len(log_data)):
        var += (log_data[i] - log_data[i-1]) * (log_data[i] - log_data[i-1])
    var /= len(log_data) * tau
    mu = XT/time_marshall_as_year
    sigma = math.sqrt(var)
    #hrst = hurst(data['Adj Close'])
    #print("mu = ", mu)
    #print("sigma = ", sigma)
    return mu, sigma

In [5]:
ticker = "MSFT"
endt = "2021-01-01"
intrvl = "1d"

tickers = ['MSFT','BRK-A','TSM','UNH','DIS','BAC','HD','PG','VZ','XOM','KO',
           'T', 'V', 'JPM','VTI','JNJ','WMT','SPY','MA','BML-PJ']
#tickers = ['TATN.ME','SNGSP.ME','SIBN.ME','YNDX.ME','PLZL.ME','LKOH.ME','GMKN.ME','NVTK.ME','GAZP.ME','ROSN.ME',
#         'NLMK.ME','MTSS.ME','CHMF.ME','ALRS.ME','VTBR.ME','FIVE.ME','IRAO.ME','MGNT.ME','POLY.ME','SBER.ME']
#tickers = ['UNH', 'T']
#st_y = ['2020-','2019-','2018-','2017-','2016-','2015-','2014-','2013-','2012-','2011-','2010-']
#st_m = ['12-','11-','10-','09-','08-','07-','06-','05-','04-','03-','02-','01-']
st_y = ['2020-']
st_m = ['12-']
para = np.zeros((len(st_y)*len(st_m),2), dtype="float32")
#getparams(ticker, strt, endt, intrvl)

para = np.zeros(len(tickers)*2, dtype="float32")
for tics in tickers:
    i = 0
    #para = np.zeros((len(st_y)*len(st_m),2), dtype="float32")

    for sy in st_y:
        for sm in st_m:
            dts = sy + sm + "31"
            print (dts)
            #para[i] = getparams(tics, dts, endt, intrvl)
            data = yf.download(tics, start=strt, end=endt, interval=intrvl)
            para[i] = getparams(tics, dts, endt, intrvl)
            i += 1   
    #print(para)
    #plt.plot(np.arange(i)+1, para[:,0])
    #plt.show()
    #plt.plot(np.arange(i)+1, para[:,1])
    #plt.show()
dt = pd.DataFrame(para)
writer = pd.ExcelWriter('C:/Users/prof-/OneDrive/Рабочий стол/ПНИПУ/Ticker_params/S0.xlsx', engine='xlsxwriter')
dt.to_excel(writer, 'Sheet1')
writer.save()

2020-12-31
year frac =  0.0027378507871321013
[*********************100%***********************]  1 of 1 completed
tau =  0.0013689253935660506
2


ValueError: setting an array element with a sequence.

In [106]:
dt = pd.DataFrame(para)
writer = pd.ExcelWriter('C:/Users/prof-/OneDrive/Рабочий стол/ПНИПУ/Ticker_params/Mu_sigma_'+ticker+'.xlsx', engine='xlsxwriter')
dt.to_excel(writer, 'Sheet1')
writer.save()